
# Coursework 2: Adversarial Search on (m, n, k)-games


For this coursework you have to implement adversarial search by Minimax and α-β pruning to
play (m, n, k)-games.
An (m, n, k)-game is a board game in which two players, Max and Min, take turns in placing Xs
(Max) and Os (Min) on an m × n board, the winner being the player who first gets k Xs (resp. Os)
in a row, horizontally, vertically, or diagonally. Thus, tic-tac-toe is the (3, 3, 3)-game.
More information can be found at https://en.wikipedia.org/wiki/M,n,k-game

### 1. [10 marks] Write a program in Python that implements adversarial search by Minimax on (m, n, k)-games. The program will include a class Game with a method play() that allows for playing the game, as well as the following:


#### (a) A constructor __init__(), a method initialize_game() to initialize the empty m × n board at the beginning, and a method drawboard() to output the board on the screen.

#### (b) At each step the program computes the Minimax strategy for Max and recommend the relevant action(s) to the user. The user is then prompted to insert the coordinates of the chosen cell. Finally, the program outputs the move for Min. Write methods max() and min() to compute Minimax values for both players, as well as methods is_valid() and is_terminal() to check for valid moves and terminal states.

#### 2. [10 marks] Implement α-β pruning when computing Minimax values to speed up action selection. In particular, modify methods max() and min() to account for the values of α and β. Again, please comment briefly your code.


#### 3. [5 marks] Check how the execution times for action selection for Minimax scale up depending on parameters m, n, and k of the game.
#### Draw an appropriate table with increasing values of m, n, and k. You can use the time module and function time() to measure the time to evaluate the game tree at every move. Is there a significant difference in selection time for actions? What about the number of visited states?


#### 4. [5 marks] Do the same as in the previous point, this time for Minimax with α-β pruning (again, you can use the time module). Is the difference with and without α-β pruning significant? Please draw a table and justify your answer with the experimental results.


### Try Codes

In [ ]:
#!/usr/bin/env python3
from math import inf as infinity
from random import choice
import platform
import time
from os import system

"""
An implementation of Minimax AI Algorithm in Tic Tac Toe,
using Python.
This software is available under GPL license.
Author: Clederson Cruz
Year: 2017
License: GNU GENERAL PUBLIC LICENSE (GPL)
"""

HUMAN = -1
COMP = +1
board = [
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
]


def evaluate(state):
    """
    Function to heuristic evaluation of state.
    :param state: the state of the current board
    :return: +1 if the computer wins; -1 if the human wins; 0 draw
    """
    if wins(state, COMP):
        score = +1
    elif wins(state, HUMAN):
        score = -1
    else:
        score = 0

    return score


def wins(state, player):
    """
    This function tests if a specific player wins. Possibilities:
    * Three rows    [X X X] or [O O O]
    * Three cols    [X X X] or [O O O]
    * Two diagonals [X X X] or [O O O]
    :param state: the state of the current board
    :param player: a human or a computer
    :return: True if the player wins
    """
    win_state = [
        [state[0][0], state[0][1], state[0][2]],
        [state[1][0], state[1][1], state[1][2]],
        [state[2][0], state[2][1], state[2][2]],
        [state[0][0], state[1][0], state[2][0]],
        [state[0][1], state[1][1], state[2][1]],
        [state[0][2], state[1][2], state[2][2]],
        [state[0][0], state[1][1], state[2][2]],
        [state[2][0], state[1][1], state[0][2]],
    ]
    if [player, player, player] in win_state:
        return True
    else:
        return False


def game_over(state):
    """
    This function test if the human or computer wins
    :param state: the state of the current board
    :return: True if the human or computer wins
    """
    return wins(state, HUMAN) or wins(state, COMP)


def empty_cells(state):
    """
    Each empty cell will be added into cells' list
    :param state: the state of the current board
    :return: a list of empty cells
    """
    cells = []

    for x, row in enumerate(state):
        for y, cell in enumerate(row):
            if cell == 0:
                cells.append([x, y])

    return cells


def valid_move(x, y):
    """
    A move is valid if the chosen cell is empty
    :param x: X coordinate
    :param y: Y coordinate
    :return: True if the board[x][y] is empty
    """
    if [x, y] in empty_cells(board):
        return True
    else:
        return False


def set_move(x, y, player):
    """
    Set the move on board, if the coordinates are valid
    :param x: X coordinate
    :param y: Y coordinate
    :param player: the current player
    """
    if valid_move(x, y):
        board[x][y] = player
        return True
    else:
        return False


def minimax(state, depth, player):
    """
    AI function that choice the best move
    :param state: current state of the board
    :param depth: node index in the tree (0 <= depth <= 9),
    but never nine in this case (see iaturn() function)
    :param player: an human or a computer
    :return: a list with [the best row, best col, best score]
    """
    if player == COMP:
        best = [-1, -1, -infinity]
    else:
        best = [-1, -1, +infinity]

    if depth == 0 or game_over(state):
        score = evaluate(state)
        return [-1, -1, score]

    for cell in empty_cells(state):
        x, y = cell[0], cell[1]
        state[x][y] = player
        score = minimax(state, depth - 1, -player)
        state[x][y] = 0
        score[0], score[1] = x, y

        if player == COMP:
            if score[2] > best[2]:
                best = score  # max value
        else:
            if score[2] < best[2]:
                best = score  # min value

    return best


def clean():
    """
    Clears the console
    """
    os_name = platform.system().lower()
    if 'windows' in os_name:
        system('cls')
    else:
        system('clear')


def render(state, c_choice, h_choice):
    """
    Print the board on console
    :param state: current state of the board
    """

    chars = {
        -1: h_choice,
        +1: c_choice,
        0: ' '
    }
    str_line = '---------------'

    print('\n' + str_line)
    for row in state:
        for cell in row:
            symbol = chars[cell]
            print(f'| {symbol} |', end='')
        print('\n' + str_line)


def ai_turn(c_choice, h_choice):
    """
    It calls the minimax function if the depth < 9,
    else it choices a random coordinate.
    :param c_choice: computer's choice X or O
    :param h_choice: human's choice X or O
    :return:
    """
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    clean()
    print(f'Computer turn [{c_choice}]')
    render(board, c_choice, h_choice)

    if depth == 9:
        x = choice([0, 1, 2])
        y = choice([0, 1, 2])
    else:
        move = minimax(board, depth, COMP)
        x, y = move[0], move[1]

    set_move(x, y, COMP)
    time.sleep(1)


def human_turn(c_choice, h_choice):
    """
    The Human plays choosing a valid move.
    :param c_choice: computer's choice X or O
    :param h_choice: human's choice X or O
    :return:
    """
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    # Dictionary of valid moves
    move = -1
    moves = {
        1: [0, 0], 2: [0, 1], 3: [0, 2],
        4: [1, 0], 5: [1, 1], 6: [1, 2],
        7: [2, 0], 8: [2, 1], 9: [2, 2],
    }

    clean()
    print(f'Human turn [{h_choice}]')
    render(board, c_choice, h_choice)

    while move < 1 or move > 9:
        try:
            move = int(input('Use numpad (1..9): '))
            coord = moves[move]
            can_move = set_move(coord[0], coord[1], HUMAN)

            if not can_move:
                print('Bad move')
                move = -1
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')


def main():
    """
    Main function that calls all functions
    """
    clean()
    h_choice = ''  # X or O
    c_choice = ''  # X or O
    first = ''  # if human is the first

    # Human chooses X or O to play
    while h_choice != 'O' and h_choice != 'X':
        try:
            print('')
            h_choice = input('Choose X or O\nChosen: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    # Setting computer's choice
    if h_choice == 'X':
        c_choice = 'O'
    else:
        c_choice = 'X'

    # Human may starts first
    clean()
    while first != 'Y' and first != 'N':
        try:
            first = input('First to start?[y/n]: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    # Main loop of this game
    while len(empty_cells(board)) > 0 and not game_over(board):
        if first == 'N':
            ai_turn(c_choice, h_choice)
            first = ''

        human_turn(c_choice, h_choice)
        ai_turn(c_choice, h_choice)

    # Game over message
    if wins(board, HUMAN):
        clean()
        print(f'Human turn [{h_choice}]')
        render(board, c_choice, h_choice)
        print('YOU WIN!')
    elif wins(board, COMP):
        clean()
        print(f'Computer turn [{c_choice}]')
        render(board, c_choice, h_choice)
        print('YOU LOSE!')
    else:
        clean()
        render(board, c_choice, h_choice)
        print('DRAW!')

    exit()


if __name__ == '__main__':
    main()


Choose X or O
Chosen: X
First to start?[y/n]: y
Human turn [X]

---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Use numpad (1..9): 1
Computer turn [O]

---------------
| X ||   ||   |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Human turn [X]

---------------
| X ||   ||   |
---------------
|   || O ||   |
---------------
|   ||   ||   |
---------------
Use numpad (1..9): 5
Bad move


In [1]:
board = [
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
]

In [2]:
board

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

[['x', 'x', 'x', 'x'],
 ['x', 'x', 'x', 'x'],
 ['x', 'x', 'x', 'x'],
 ['x', 'x', 'x', 'x']]

In [33]:
for row_count,row in enumerate(board):
    if row_count !=0:
        print(f'\n{"_"*10}')
    for col_count,col in enumerate(row):
        if col ==None:
            print(' ',end="")
        else:
            print(col,end="")
        if col_count==2:
            pass
        else:
            print(" | ",end="")

x | x | xx | 
__________
x | x | xx | 
__________
x | x | xx | 
__________
x | x | xx | 

In [40]:
def format_row(row):
    return '|' + '|'.join('{0:^3s}'.format(x) for x in row) + '|'

def format_board(board):

    return '\n\n'.join(format_row(row) for row in board)

In [47]:
m=3 #columns
n=3 #rows
board=[["x" for i in range(m)] for i in range (n)]
board

print(format_board(board))

| x | x | x |

| x | x | x |

| x | x | x |


In [57]:
self_m=3 #columns
self_n=3 #rows
array_board=[[' ' for i in range(self_m)] for i in range(self_n)]
array_board

[[' ', ' ', ' '], [' ', ' ', ' '], [' ', ' ', ' ']]

In [53]:
UPPER_CASE_OFFSET=64

array_first_line=[chr(code + UPPER_CASE_OFFSET) for code in range(1, self_m + 1)]
array_first_line

['A', 'B', 'C']

In [55]:

first_line =' ' * (len(str(self_n)) + 3) +(' ' * 3).join(array_first_line) + ' \n'
first_line

'    A   B   C \n'

In [61]:
list_lines=[]
for index_line, array_line in enumerate(array_board, 1):
    index_line = self_n + 1 - index_line
    number_spaces_before_line = len(str(self_n)) - len(str(index_line))
    space_before_line = number_spaces_before_line * ' '
    list_lines.append(f'{space_before_line}{index_line} | ' +' | '.join(array_line) + f' | {index_line}\n')
list_lines

['3 |   |   |   | 3\n', '2 |   |   |   | 2\n', '1 |   |   |   | 1\n']

In [63]:
line_dashes = (len(str(self_n)) + 1)*' ' + '-' * 4 * self_m + '-\n'
line_dashes

'  -------------\n'

In [66]:
board_str = first_line + line_dashes + line_dashes.join(list_lines) +\
                    line_dashes + first_line
print(board_str)

    A   B   C 
  -------------
3 |   |   |   | 3
  -------------
2 |   |   |   | 2
  -------------
1 |   |   |   | 1
  -------------
    A   B   C 



In [71]:
x=True
y=True
z=True
w=False

(x==y==z==w==True)

False